# Import libraries

In [5]:
## remove this, this is for my personal pc setup
import sys; import os; sys.path.append(os.path.expanduser('~/Google Drive/my/projects/t/'))

In [7]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import pandas   as pd
import operator as op

import numpy             as np
import seaborn           as sns
from datetime import datetime

from IPython.display import IFrame

import t as t # want T to be accessible


%load_ext autoreload
%autoreload

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from fbprophet import Prophet
# https://twitter.com/seanjtaylor/status/1123278380369973248

# Read in the data

Read the data in from the retail sales CSV file in the examples folder then set the index to the 'date' column. We are also parsing dates in the data file.

In [ ]:
coin_df = pd.read_csv('../_data/bitcoin_price.csv', index_col='Date', parse_dates=True)

In [ ]:
coin_df.head()

# Prepare for Prophet

For prophet to work, we need to change the names of these columns to 'ds' and 'y', so lets just create a new dataframe and keep our old one handy (you'll see why later). The new dataframe will initially be created with an integer index so we can rename the columns

In [ ]:
df = coin_df.reset_index()

In [ ]:
df.head()

Let's rename the columns as required by fbprophet. Additioinally, fbprophet doesn't like the index to be a datetime...it wants to see 'ds' as a non-index column, so we won't set an index differnetly than the integer index.

In [ ]:
df=df.rename(columns={'Date':'ds', 'Open':'y'})

In [ ]:
df.head()

Now's a good time to take a look at your data.  Plot the data using pandas' ```plot``` function

In [ ]:
df.set_index('ds').y.plot()

# Running Prophet

Now, let's set prophet up to begin modeling our data.

Note: Since we are using daily data, you'll see a message from Prophet saying ```Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.```  This is OK since we are workign with daily data but you can disable it by using ```daily_seasonality=True``` in the instantiation of Prophet.

In [ ]:
model = Prophet()
model.fit(df);

Forecasting is fairly useless unless you can look into the future, so we need to add some future dates to our dataframe. For this example, I want to forecast 2 years into the future, so I'll built a future dataframe with 24 periods since we are working with monthly data. Note the ```freq='m'``` inclusion to ensure we are adding 24 months of data.

This can be done with the following code:


In [ ]:
future = model.make_future_dataframe(periods=90, freq = 'd')
future.tail()

To forecast this future data, we need to run it through Prophet's model.

In [ ]:
forecast = model.predict(future)

The resulting forecast dataframe contains quite a bit of data, but we really only care about a few columns.  First, let's look at the full dataframe:

In [ ]:
forecast.tail()

We really only want to look at yhat, yhat_lower and yhat_upper, so we can do that with:

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# Plotting Prophet results

Prophet has a plotting mechanism called ```plot```.  This plot functionality draws the original data (black dots), the model (blue line) and the error of the forecast (shaded blue area).

In [ ]:
model.plot(forecast)

In [4]:
model.plot_components(forecast);

NameError: name 'model' is not defined